# Binary Classification

In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Device Agnostic Code

In [85]:
device = "cuda" if torch.cuda.is_available() else "cpu"

device

'cpu'

### Data PreProcessing

In [86]:
data = pd.read_csv('./dataset-classification/heart.csv')
data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [87]:
data = data[['Age', 'Sex', 'RestingBP', 'Cholesterol', 'MaxHR', 'HeartDisease']]
data.head()

,Age,Sex,RestingBP,Cholesterol,MaxHR,HeartDisease
0,40,M,140,289,172,0
1,49,F,160,180,156,1
2,37,M,130,283,98,0
3,48,F,138,214,108,1
4,54,M,150,195,122,0


In [88]:
X = data[['Age', 'Sex', 'RestingBP', 'Cholesterol', 'MaxHR']]
y = data['HeartDisease']

y.head()

0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64

In [89]:
# convert sex to numerical value

X = X.replace({'M':1, 'F':0})

X.head()

/tmp/ipykernel_387719/1854787251.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X = X.replace({'M':1, 'F':0})


,Age,Sex,RestingBP,Cholesterol,MaxHR
0,40,1,140,289,172
1,49,0,160,180,156
2,37,1,130,283,98
3,48,0,138,214,108
4,54,1,150,195,122


In [90]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(X)

features_scaled

array([[-1.4331398 ,  0.51595242,  0.41090889,  0.82507026,  1.38292822],
       [-0.47848359, -1.93816322,  1.49175234, -0.17196105,  0.75415714],
       [-1.75135854,  0.51595242, -0.12951283,  0.7701878 , -1.52513802],
       ...,
       [ 0.37009972,  0.51595242, -0.12951283, -0.62016778, -0.85706875],
       [ 0.37009972, -1.93816322, -0.12951283,  0.34027522,  1.4615246 ],
       [-1.64528563,  0.51595242,  0.30282455, -0.21769643,  1.42222641]])

### Split the data

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

### Convert Dataframes to Tensors

In [92]:
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

In [93]:
X_train = torch.tensor(X_train_np, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test_np, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train_np, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test_np, dtype=torch.float32).to(device)

In [94]:
X_train.shape, y_train.shape, X_train.dim(), y_train.dim()

(torch.Size([780, 5]), torch.Size([780]), 2, 1)

In [95]:
X_train[:5], y_train[:5]

(tensor([[ 48.,   1., 110., 229., 168.],
         [ 55.,   1., 140., 217., 111.],
         [ 42.,   1., 160., 147., 146.],
         [ 63.,   1., 100.,   0., 109.],
         [ 62.,   1., 160., 254., 108.]]),
 tensor([1., 1., 0., 1., 1.]))

### Creating Model

In [100]:
class myModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=5, out_features=10)
        self.layer2 = nn.Linear(in_features=10, out_features=12)
        self.layer3 = nn.Linear(in_features=12, out_features=1)
        self.relu = nn.ReLU
    
    def forward(self, x):
        return self.layer3(self.relu(self.layer2(self.relu(self.layer1(x)))))

In [101]:
model = myModel().to(device)

model

myModel(
  (layer1): Linear(in_features=5, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=12, bias=True)
  (layer3): Linear(in_features=12, out_features=1, bias=True)
  (relu): ReLU()
)

### Setup Loss Function and Optimizer

In [102]:
loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

### Training the model